In [1]:
from reservoir_rls_multires import *
import matplotlib.pyplot as plt
from lorenz63 import *
from scipy.signal import welch, periodogram
from sklearn.preprocessing import StandardScaler
import cma

In [2]:
get_data = False
data_length = 1000000
step = 0.05
f_s = 1/step
scale = 0.01
slow_var = 48/28
r_t = lambda x: r_t_const(x)
dx_dt = lambda x, time, r_t: dxdt_lorenz_rossler(x, time, r_t, scale = scale, slow_var = slow_var)
if get_data:
    lorenz_data_rossler = getCoupledLorenzData(data_length, r_t, dx_dt, sample_tau = step)
    np.savetxt('lorenz_data_rossler_step%0.2f_scale%0.2f.csv' %(step, scale), lorenz_data_rossler, delimiter = ',')
else:
    lorenz_data_rossler = np.loadtxt('lorenz_data_rossler_step%0.2f_scale%0.2f.csv' %(step, scale), delimiter = ',')
"""
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(111,projection = '3d')
ax.plot3D(lorenz_data_rossler[:,0],lorenz_data_rossler[:,1],lorenz_data_rossler[:,2])
plt.show()
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(111,projection = '3d')
ax.plot3D(lorenz_data_rossler[:,3],lorenz_data_rossler[:,4],lorenz_data_rossler[:,5])
plt.show()
"""
init_delay = 10000
scaled_data = lorenz_data_rossler[init_delay:,:3]
scaled_data = np.ascontiguousarray(scaled_data)
SS = StandardScaler()
"""
scaled_data_external = SS.fit_transform(lorenz_data_rossler[:,4].reshape(-1,1))
data_external = lorenz_data_rossler[:,4]

Wn_xy = 0.0075
Wn_z = Wn_xy
lorenz_data_split = separate_lorenz_2scale(lorenz_data_rossler[:,:3], f_s, Wn_xy, Wn_z, filter_order = 10)
data_external = lorenz_data_split[init_delay:,5]
lorenz_data_rossler  = lorenz_data_rossler[init_delay:]
scaled_data_external = SS.fit_transform(data_external.reshape(-1,1))
plt.plot(lorenz_data_rossler[:,4])
plt.plot(data_external-np.mean(data_external))
plt.show()
"""

'\nscaled_data_external = SS.fit_transform(lorenz_data_rossler[:,4].reshape(-1,1))\ndata_external = lorenz_data_rossler[:,4]\n\nWn_xy = 0.0075\nWn_z = Wn_xy\nlorenz_data_split = separate_lorenz_2scale(lorenz_data_rossler[:,:3], f_s, Wn_xy, Wn_z, filter_order = 10)\ndata_external = lorenz_data_split[init_delay:,5]\nlorenz_data_rossler  = lorenz_data_rossler[init_delay:]\nscaled_data_external = SS.fit_transform(data_external.reshape(-1,1))\nplt.plot(lorenz_data_rossler[:,4])\nplt.plot(data_external-np.mean(data_external))\nplt.show()\n'

In [3]:
def phase_min_func(delay, truth, filtered):
    delay = round(delay)
    truth_delayed = truth[:truth.shape[0]-delay]
    filtered_delayed = filtered[delay:]
    val = -np.mean(truth_delayed * filtered_delayed)
    return val
def min_func_wphase(x, mask, base_data, f_s, true_external_data,\
    base_res, num_tests, num_nodes, pred_length, train_lenght, scale = True, 
    external_output = True):
    init_delay = 20000
    max_freq = 0.15
    min_freq = 0.001
    Wn_xy = x[0]/10*(max_freq-min_freq)+min_freq
    Wn_z = Wn_xy
    data_split = separate_lorenz_2scale(base_data, f_s, Wn_xy, Wn_z, filter_order = 10)
    base_external_data = data_split[init_delay:,-1]
    num_delays = 10000
    z_centered = base_external_data - np.mean(base_external_data)
    min_func   = lambda delay: phase_min_func(delay, true_external_data[init_delay:], z_centered)
    func_vals = np.zeros(num_delays)
    for i in range(num_delays):
        func_vals[i] = min_func(i)
    min_delay = np.argmin(func_vals)
    data = base_data[init_delay:base_data.shape[0]-min_delay]
    external_data = base_external_data[min_delay:]
    if scale:
        SS = StandardScaler()
        external_data = SS.fit_transform(external_data.reshape(-1,1))
    funval = vt_min_function_norm_external(data,external_data, x[1:], mask, base_res.Win, base_res.A, \
        num_tests = num_tests,  num_nodes = num_nodes, pred_length = pred_length, train_length = train_length,\
        external_output = external_output)
    return funval
num_nodes = 360
num_tests = 200
train_length = 3000
sync_length = 500
pred_length = 500
res_seed = 1
base_res = reservoir(4,num_nodes,input_weight = 1, spectral_radius = 1, seed = res_seed) #Generate a reservoir
mask = ['input_weight', 'regularization', 'leakage', 'forget']
x0 = np.array([1,6,4,0,9])
min_func_base = lambda x: min_func_wphase(x, mask, np.ascontiguousarray(lorenz_data_rossler[:,:3]), f_s, lorenz_data_rossler[:,4],\
    base_res, num_tests, num_nodes, pred_length, train_length, scale = False, external_output = False)
sigma = 2

In [4]:
opts = cma.CMAOptions()
opts.set('popsize',10*x0.size) # Set number of samples per generation
"""
Set bounds on parameters. IMPORTANT: The mean returned by cma-es is
the mean BEFORE the boundary function is applied, so the mean may not
lie in the domain set by bounds. To obtain the true sample mean requires 
downloading the cma-es package from github and editing one of the 
functions. Ask me if you need to do this.
"""
opts.set('bounds', [0,10]) 
opts.set('seed', 5) # Seed for the initial samples
"""
File where results are saved. IMPORTANT: Full covariance matrix is 
NOT saved, nor are the exact samples. If these need to be saved, one
will also have to download from github and make some edits. Again,
ask me.
"""
opts.set('verb_filenameprefix','cmaes_lorenz_rossler_wextern_res%d\\' % res_seed)
results = cma.fmin(min_func_base, x0, sigma, options = opts) # Run the algorithm

(25_w,50)-aCMA-ES (mu_w=14.0,w_1=14%) in dimension 5 (seed=5, Thu Jan 21 13:45:57 2021)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     50 -2.550000000000000e+01 1.0e+00 2.04e+00  2e+00  2e+00 29:35.7
    2    100 -3.000000000000000e+01 1.2e+00 2.06e+00  1e+00  2e+00 60:41.7
    3    150 -3.100000000000000e+01 1.5e+00 1.99e+00  1e+00  2e+00 91:14.1
    4    200 -3.150000000000000e+01 1.9e+00 1.90e+00  7e-01  2e+00 121:16.4


LinAlgError: Matrix is singular to machine precision.